<a href="https://colab.research.google.com/github/FedorTaggenbrock/data_intensive_systems/blob/main/notebooks/main_tests_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python ../main_tests.py

test


**Pip install stuff that needs to be installed before importing anything**  
(for colab only; cannot be done inside the modules, would be stupid)

In [2]:
import sys
ON_COLAB = 'google.colab' in sys.modules
if ON_COLAB:
    # Do stuff that only needs to happen on colab
    !pip install pyspark
else:
    # Do stuff that only needs to happen on local computer
    pass

In [4]:
if ON_COLAB:
    print("Running in Colab: importing from GitHub.")
    # wget ("curl") all files from github
    user = 'FedorTaggenbrock'
    repo = 'data_intensive_systems'
    src_dir = "src"
    pyfiles = ["clustering.py"]

    for pyfile in pyfiles:
        url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{src_dir}/{pyfile}"
        !wget --no-cache --backups=1 {url}

    # Import all files
    try:
        import clustering as clustering
    except Exception as e:
        print('Error importing files on colab. Are the names right?.')

else:
    print("Running locally, importing from local.")
    # Append parent folder to path
    sys.path.append('..')
    # And import all files
    from src import clustering as clustering


Running locally, importing from local.


**Test if this works**


In [6]:
# print(dir(clustering))
res = clustering.jaccard_distance([1,1,0,1,0], [1,1,1,1,0])
print(res)
clustering.clustering_test1()

0.25
Initialized Spark. Start clustering.
centroids =  [(1, 1, 0, 1, 0), (1, 0, 0, 0, 1)]
centroids =  [(1, 1, 0, 1, 0), (0, 0, 1, 0, 1)]
Finished clustering. Start evaluation.
[1, 1, 0, 1, 0]
[0, 0, 1, 0, 1]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 6.0 failed 1 times, most recent failure: Lost task 1.0 in stage 6.0 (TID 25) (145.136.150.11 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Program Files\spark-3.4.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 830, in main
  File "C:\Program Files\spark-3.4.0-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 820, in process
  File "c:\Python\lib\site-packages\pyspark\rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\Python\lib\site-packages\pyspark\rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\Python\lib\site-packages\pyspark\rdd.py", line 828, in func
    return f(iterator)
  File "c:\Python\lib\site-packages\pys

**Create a spark instance**

In [ ]:
spark_settings = {}
spark_instance = make_spark()

**Load and parse data**

In [ ]:
# Load and parse data
data_for_spark = parse_data()

In [ ]:
# Run clustering and parameter tuning
clustering_settings = {}
clustering_centroid_outcomes = run_clustering(spark_instance, data_for_spark, clustering_settings)

In [ ]:
# Evaluate clustering results
clustering_evaluation = evaluate_clustering(clustering_centroid_outcomes)

In [ ]:
# Display results (maybe only best result)
clustering_result = display_results(clustering_evaluation